# ProcessCcd 

Owner(s): **Alex Drlica-Wagner (@kadrlica)**

Last Verified to Run: **2018-08-10**

Verified Stack Release: **16.0**

This notebook seeks to mostly replicate the first two steps of the LSST Stack ["Getting started tutorials"](https://pipelines.lsst.io/getting-started/index.html#getting-started-tutorials). We first setup a working directory to point to an installation of the HSC data (more details on available data sets can be found in [DataInventory.ipynb](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/Basics/DataInventory.ipynb)).

In [ ]:
!mkdir DATA

In [ ]:
!echo "lsst.obs.hsc.HscMapper" > DATA/_mapper

In [ ]:
!ingestImages.py DATA /project/shared/data/ci_hsc/raw/*.fits --mode=link

In [ ]:
!installTransmissionCurves.py DATA

In [ ]:
!ln -s /project/shared/data/ci_hsc/CALIB/ DATA/CALIB

In [ ]:
!mkdir -p DATA/ref_cats
!ln -s /project/shared/data/ci_hsc/ps1_pv3_3pi_20170110 DATA/ref_cats/ps1_pv3_3pi_20170110

In [ ]:
!processCcd.py DATA --rerun processCcdOutputs --id --show data

In [ ]:
!processCcd.py DATA --rerun processCcdOutputs --id filter=HSC-I --show data

In [ ]:
!processCcd.py DATA --rerun processCcdOutputs --id